# Praxis: Fitting power-laws to data
In this assigment we will be combining what we learned this week to fit a power-law to some data and then visualize both the power-law and the fit.

## (1) Getting the data
First we need to generate some data to fit to. We will use the degree distribution from the Barabasi-Albert graph because we know that it is power-law distributed. Create a BA graph using networkx with `n = 2500` and `m = 8` (adjust `n` or `m` if your computer has difficulty creating the graph ).

Then build a list/array of the node degrees.

In [ ]:
import networkx as nx
import powerlaw
import numpy as np

sf_graph = nx.barabasi_albert_graph(2500, 8, seed=8)
degree_distribution = np.array([ sf_graph.degree(node) for node in sf_graph ])

## (2) Find the power-law cut-off
It is normally the case in power-law distributed data that there is a value below which the power-law relation does not hold. We will express this value as `xmin`. In order to fit the scaling exponent, we first need to estimate `xmin` and then discard all values in the distribution below it.

Since calculating the optimal `xmin` can be a challenge in itself, you can use Aaron Clauset's [powerlaw](http://tuvalu.santafe.edu/~aaronc/powerlaws/) package to determine `xmin`. The package can be installed using pip from the commandline: `pip install powerlaw`

The package should then be available for import:

```
import powerlaw
```

And can be called using (see [documentation for additional arguments](http://arxiv.org/pdf/1305.0215v3.pdf)):

```
fit = powerlaw.Fit(some_data) 
print(fit.xmin)
```

Alternatively, you can get a rough estimate of the cut-off yourself by visualizing the power-law.

In [ ]:
fit = powerlaw.Fit(degree_distribution)
xmin = fit.xmin
print(xmin)

## (3) Find scaling exponent
With the data available and `xmin` estimated we can now estimate the scaling exponent for our degree distribution. Use the maximum-likelihood approach to estimate the scaling exponent (see equation 3.7 from [the reading](https://iu.instructure.com/courses/1564562/files/64585579?module_item_id=15083021)).

In [ ]:
def discrete_estimator(distribution, cutoff):
    cut_distribution = distribution[distribution > cutoff ]
    return 1 + len(cut_distribution) / np.sum([ np.log(val / (cutoff - 0.5)) 
                                           for val in cut_distribution ])

scaling_exp = discrete_estimator(degree_distribution, xmin)
print(scaling_exp)
print(fit.power_law.alpha)
print(fit.power_law.xmin)

## (4) Visualize the power-law
Plot the CCDF of the power-law along with a best-fit line made using your estimated scaling exponent. If the fit is good, it should fall right on-top of the data-points on the CCDF plot.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
def ecdf(data):

    data = np.sort(data)
    size = float(len(data))
    all_unique = not( any( data[:-1]==data[1:] ) )
    if all_unique:
        cdf = np.array([ i / size for i in range(0,len(data)) ])
    else:
        cdf = np.searchsorted(data, data,side='left')/size
        unique_data, unique_indices = np.unique(data, return_index=True)
        data=unique_data
        cdf = cdf[unique_indices]

    return data, cdf

def eccdf(data):

    sorted_data, cdf = ecdf(data)
    return sorted_data, 1. - cdf

def plot_ccdf(data, xlabel='', x_log=False, y_log=False):

    x, y = eccdf(data)
    plt.clf()
    plt.plot(x, y, 'b-')
    if x_log == True: plt.xscale('log')
    if y_log == True: plt.yscale('log')
    plt.ylabel('CCDF')
    plt.xlabel('degree')
    plt.show()

In [ ]:
plot_ccdf(degree_distribution, x_log=True, y_log=True)

In [ ]:
def plot_ccdf(data, alpha, x_min, x_log=False, y_log=False):

    # Empirical
    x, y = eccdf(data)
    
    # Model
    cut_x = x[x > x_min]
    my = np.power(x / x_min, -alpha+1)
    
    plt.clf()
    plt.plot(x, y, 'bo')
    plt.plot(x, my, ls='-', color='red', lw=2)
    if x_log == True: plt.xscale('log')
    if y_log == True: plt.yscale('log')
    plt.ylabel('CCDF')
    plt.xlabel('degree')
    plt.show()
    
plot_ccdf(degree_distribution, scaling_exp, xmin, x_log=True, y_log=True)